In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install pandera==0.14.0

Reason for being yanked: missing dependency
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.3/148.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: pandera
    Found existing installation: pandera 0.21.0
    Uninstalling pandera-0.21.0:
      Successfully uninstalled pandera-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
recommenders 1.2.0 requires pandera[strategies]>=0.15.0; python_version >= "3.9", but you have pandera 0.14.0 which is incompatible.


In [4]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
from recommenders.evaluation.python_evaluation import (
    map, ndcg_at_k, precision_at_k, recall_at_k
)
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.utils.notebook_utils import store_metadata

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


System version: 3.10.12 (main, Sep 11 2024, 15:47:36) [GCC 11.4.0]
Pandas version: 2.2.2
Tensorflow version: 2.17.1


In [5]:
df_clicks = pd.read_csv("/content/drive/MyDrive/yoochoose-clicks-100k.csv")

In [6]:
df_test = pd.read_csv("/content/drive/MyDrive/yoochoose-test-100k.csv")

In [7]:
data = pd.concat([df_clicks,df_test])

In [8]:
data['rating'] = 1

In [9]:
data.rename(columns={'SessionId': 'userID', 'ItemId': 'itemID'}, inplace=True)

In [10]:
# Create a list of all possible user-item pairs
all_items = data['itemID'].unique()
users = data['userID'].unique()

negative_samples = []
for user in users:
    interacted_items = data[data['userID'] == user]['itemID'].unique()
    non_interacted_items = np.setdiff1d(all_items, interacted_items)
    sampled_items = np.random.choice(non_interacted_items, size=10, replace=False)
    negative_samples.extend([(user, item, 0) for item in sampled_items])

In [11]:
negative_samples = pd.DataFrame(negative_samples, columns=['userID', 'itemID', 'rating'])

In [12]:
final_data = pd.concat([data[['userID', 'itemID', 'rating']], negative_samples])

In [13]:
train = final_data.groupby('userID').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
test = final_data.groupby('userID').apply(lambda x: x.iloc[-1]).reset_index(drop=True)

<ipython-input-13-371e47ca4c3c>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train = final_data.groupby('userID').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
<ipython-input-13-371e47ca4c3c>:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test = final_data.groupby('userID').apply(lambda x: x.iloc[-1]).reset_index(drop=True)


In [14]:
test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

In [15]:
leave_one_out_test = test.groupby("userID").last().reset_index()

In [16]:
train_file = "./train.csv"
test_file = "./test.csv"
leave_one_out_test_file = "./leave_one_out_test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)
leave_one_out_test.to_csv(leave_one_out_test_file, index=False)

In [17]:
SEED = DEFAULT_SEED

In [18]:
data = NCFDataset(train_file=train_file, test_file=test_file, seed=42)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/usr/local/lib/py

In [ ]:
import time
from recommenders.models.ncf.ncf_singlenode import NCF

model = NCF(
    n_users=data.n_users,
    n_items=data.n_items,
    model_type="NeuMF",
    n_factors=64,
    layer_sizes=[8, 4, 2],
    n_epochs=10,
    batch_size=50,
    learning_rate=0.01,
    verbose=10,
    seed=42,
)

model.fit(data)




/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


In [ ]:
with Timer() as train_time:
    model.fit(data)

print("Took {} seconds for training.".format(train_time.interval))

In [ ]:
from tqdm import tqdm

def evaluate_model(model, train, test, K=10):
    hits = []
    ndcgs = []
    all_items = final_data['itemID'].unique()

    for user in tqdm(test['userID'].unique()):
        test_item = test[test['userID'] == user]['itemID'].values[0]
        train_items = train[train['userID'] == user]['itemID'].unique()
        non_interacted_items = np.setdiff1d(all_items, train_items)

        # Ensure there are enough items to sample
        if len(non_interacted_items) >= 99:
            negative_items = np.random.choice(non_interacted_items, size=99, replace=False)
        else:
            negative_items = non_interacted_items

        items = np.append(negative_items, test_item)

        # Prepare user and item inputs for prediction
        users = np.full(len(items), user)
        users = users.astype(np.int32)
        items = items.astype(np.int32)

        # Get prediction scores
        predictions = model.predict(users, items, is_list=True)
        item_score_map = {item: score for item, score in zip(items, predictions)}

        # Rank items based on predictions
        ranked_items = sorted(item_score_map.items(), key=lambda x: x[1], reverse=True)
        ranked_items = [x[0] for x in ranked_items]

        # Calculate rank of test item
        try:
            rank = ranked_items.index(test_item) + 1  # Ranks start at 1
        except ValueError:
            rank = None  # Test item not in ranked list

        # Calculate Hit Ratio and NDCG
        if rank is not None and rank <= K:
            hits.append(1)
            ndcgs.append(1 / np.log2(rank + 1))
        else:
            hits.append(0)
            ndcgs.append(0)

    hr = np.mean(hits)
    ndcg = np.mean(ndcgs)
    return hr, ndcg

# Evaluate the model
hr10, ndcg10 = evaluate_model(model, train, test, K=10)
print(f"HR@10: {hr10}")
print(f"NDCG@10: {ndcg10}")
